# 目次
1. フォロワーの取得
- idからユーザー情報を取得
- フォローリストの取得

## フォロワーの取得

In [1]:
import json
import time
import random

import numpy as np
import pandas as pd
from requests_oauthlib import OAuth1Session

In [5]:
# APIの取得
def getAPI():
    # キー取得
    with open('../data/key.json') as f:
        keys = json.load(f)
    
    # 環境変数から承認情報を取得
    CONSUMER_KEY    = keys['CONSUMER_KEY']
    CONSUMER_SECRET = keys['CONSUMER_SECRET']
    ACCESS_TOKEN    = keys['ACCESS_TOKEN']
    ACCESS_SECRET   = keys['ACCESS_SECRET']
    api = OAuth1Session(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET)
    return api

In [23]:
# 特定ユーザーフォロワーidリストを取得
def getFollowerIds(api, screen_name):
    ids = []
    cursor = -1
    
    url = 'https://api.twitter.com/1.1/followers/ids.json'
    params = {'screen_name': screen_name}
    req = api.get(url, params=params)
    
    
    if req.status_code == 200:
        while cursor != 0:
            params['cursor'] = str(cursor)
            temp = json.loads(req.text)

            ids.extend(temp['ids'])
            print(len(temp['ids']))
            
            print("ids[-1]:{}".format(ids[-1]))
            cursor = temp['next_cursor']
            print("cursor:{}".format(cursor))
            time.sleep(10*random.uniform(0.5,1.5))
            params = {'screen_name': screen_name,'cursor':cursor}
            req = api.get(url, params=params)
            
    else:
        print ("Error: %d at getFollowerIds" % req.status_code)
    return ids

In [24]:
api = getAPI()

ids = getFollowerIds(api, screen_name='henjinmajime')
len(ids)

5000
ids[-1]:2274411320
cursor:1596258388578586144
5000
ids[-1]:937188233544212480
cursor:1591762575901258273
5000
ids[-1]:924342342663921664
cursor:1585206745275544247
5000
ids[-1]:917162648168415234
cursor:1580732814817877143
4319
ids[-1]:200719204
cursor:0


24319

## idからユーザー情報を取得
- 取得情報  
user_id/screen_name/num_friend/num_follower/self_description

In [8]:
# idから上記の情報を取得
def getUserInfo(api, ids):
    url = 'https://api.twitter.com/1.1/users/lookup.json'
    ids = np.array(ids).astype(str)
    user_info = []
    
    for i in range(0, len(ids), 100):
        _ids = ','.join(ids[i:i+100])
        
        params = {'user_id': _ids}
        
        req = api.get(url, params=params)
        if req.status_code == 200:
            req_text = json.loads(req.text)
            user_info.extend([[user['id'], user['screen_name'], user['friends_count'], user['followers_count'], user['description']] for user in req_text])
        else:
            print ("Error: %d at getUserInfo()" % req.status_code)
        time.sleep(10*random.uniform(0.5,1.5))
    return user_info

In [ ]:
user_info = getUserInfo(api, ids)

In [ ]:
df = pd.DataFrame(user_info, columns=['user_id', 'screen_name', 'num_friends', 'num_followers', 'self_description'])

In [ ]:
df.to_csv('./data/user_info.csv', index=False)

In [ ]:
# csv出力する
df = pd.read_csv('./data/user_info.csv')

In [ ]:
df

## フォローリストの取得

In [9]:
# 特定ユーザーフォローidリストを取得
def getFriendIds(api, user_id):
    ids = []
    cursor = -1
    
    url = 'https://api.twitter.com/1.1/friends/ids.json'
    params = {'user_id': user_id}
    req = api.get(url, params=params)
    
    if req.status_code == 200:
        while cursor != 0:
            params['cursor'] = str(cursor)
            temp = json.loads(req.text)

            ids.extend(temp['ids'])
            cursor = temp['next_cursor']
            time.sleep(10*random.uniform(0.5,1.5))
    else:
        print ("Error: %d at getFollowerIds" % req.status_code)
    return ids

In [ ]:
import psycopg2
class psql_save():
    def initial_section():
        self.conn = psycopg2.connect(
        host="localhost",
        database="twitter_analytics",
        port="5432",
        user="kyohei",
        password="password"
        )
        self.cursor = self.conn.cursor()
        self.cursor.execute('''
                CREATE TABLE IF NOT EXISTS followers (
                id INT,
                screen_name TEXT,
                num_friends INT,
                num_followers INT,
                self_discription TEXT,
                PRIMARY KEY(id)
                )
            ''')
        self.conn.commit()

    def process_section(item):
        self.cursor.execute('''
              INSERT INTO followers VALUES (%d, %s, %d, %d, %s)''',
               (item['title'],
                item['url'],
                item['date'],
                item['address'],
                item['img_url']
                ))
        self.conn.commit()

